## Setting



In [1]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 5.5 MB/s 
     |████████████████████████████████| 16.7 MB 234 kB/s 
     |████████████████████████████████| 6.3 MB 15.0 MB/s 


In [2]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import geopandas as gpd
import time
import json
import folium
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import seaborn as sns
import fiona
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry import Point, Polygon, LineString
from shapely import wkt

from tqdm import trange, tqdm, tqdm_notebook, tnrange

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import Binarizer, MinMaxScaler
import sklearn.metrics as metrics
from lightgbm import LGBMClassifier
import xgboost as xgb

from imblearn.over_sampling import SMOTE
from functools import partial
#from bayes_opt import BayesianOptimization

In [3]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

## data 불러오기

In [4]:
cd /content/drive/MyDrive/Colab Notebooks/22-1boaz/방학분석미니플젝/data_final

/content/drive/MyDrive/Colab Notebooks/22-1boaz/방학분석미니플젝/data_final


In [5]:
train_fast = pd.read_csv('train_fast_final.csv')
test_fast = pd.read_csv('test_fast_final.csv')

train_slow = pd.read_csv('train_slow_final.csv')
test_slow = pd.read_csv('test_slow_final.csv')

In [6]:
train_fast_X = train_fast.iloc[:, :-1]
train_fast_y = train_fast.iloc[:, -1]
test_fast_X = test_fast.iloc[:, :-1]
test_fast_y = test_fast.iloc[:, -1]

train_slow_X = train_slow.iloc[:, :-1]
train_slow_y = train_slow.iloc[:, -1]
test_slow_X = test_slow.iloc[:, :-1]
test_slow_y = test_slow.iloc[:, -1]

In [7]:
 # (102316, 67) 급속
train_fast_X.shape, train_fast_y.shape, test_fast_X.shape, test_fast_y.shape

((102316, 67), (102316,), (25575, 67), (25575,))

In [8]:
train_slow_X.shape, train_slow_y.shape, test_slow_X.shape, test_slow_y.shape

((102294, 67), (102294,), (25575, 67), (25575,))

## LightGBM 모델 적합

In [9]:
lgbm_clf_fast = LGBMClassifier(n_jobs=-1)
lgbm_clf_slow = LGBMClassifier(n_jobs=-1)

In [10]:
model_fast = [lgbm_clf_fast]
model_slow = [lgbm_clf_slow]

In [11]:
for x in model_fast:
    x.fit(train_fast_X, train_fast_y)
for x in model_slow:
    x.fit(train_slow_X, train_slow_y)

## 성능평가

In [12]:
thresholds = np.arange(0,1,0.1)

for x in model_fast:
    print(x)
    pred_proba=x.predict_proba(np.array(test_fast_X))
    pred_proba_1 = pred_proba[:,1].reshape(-1,1)
    for threshold in thresholds:
        binarizer = Binarizer(threshold=threshold)
        model_cl_pred = binarizer.transform(pred_proba_1)
        print('threshold : ', threshold)
        print('오차 행렬')
        confusion = confusion_matrix(test_fast_y,model_cl_pred)
        print(confusion)
        print('accuracy : ', accuracy_score(test_fast_y,model_cl_pred))
        print('precision : ', precision_score(test_fast_y,model_cl_pred))
        print('recall : ', recall_score(test_fast_y,model_cl_pred))
        print('f1_score : ', f1_score(test_fast_y,model_cl_pred))
        print('')
    print('===================================================')
    print('')

LGBMClassifier()
threshold :  0.0
오차 행렬
[[    0 25550]
 [    0    25]]
accuracy :  0.0009775171065493646
precision :  0.0009775171065493646
recall :  1.0
f1_score :  0.001953125

threshold :  0.1
오차 행렬
[[25534    16]
 [    6    19]]
accuracy :  0.9991397849462366
precision :  0.5428571428571428
recall :  0.76
f1_score :  0.6333333333333332

threshold :  0.2
오차 행렬
[[25536    14]
 [    7    18]]
accuracy :  0.9991788856304985
precision :  0.5625
recall :  0.72
f1_score :  0.631578947368421

threshold :  0.30000000000000004
오차 행렬
[[25539    11]
 [    8    17]]
accuracy :  0.9992570869990225
precision :  0.6071428571428571
recall :  0.68
f1_score :  0.6415094339622641

threshold :  0.4
오차 행렬
[[25540    10]
 [    8    17]]
accuracy :  0.9992961876832844
precision :  0.6296296296296297
recall :  0.68
f1_score :  0.6538461538461539

threshold :  0.5
오차 행렬
[[25543     7]
 [    9    16]]
accuracy :  0.9993743890518084
precision :  0.6956521739130435
recall :  0.64
f1_score :  0.6666666666666666

In [13]:
thresholds = np.arange(0,1,0.1)

for x in model_slow:
    print(x)
    pred_proba=x.predict_proba(np.array(test_slow_X))
    pred_proba_1 = pred_proba[:,1].reshape(-1,1)
    for threshold in thresholds:
        binarizer = Binarizer(threshold=threshold)
        model_cl_pred = binarizer.transform(pred_proba_1)
        print('threshold : ', threshold)
        print('오차 행렬')
        confusion = confusion_matrix(test_slow_y,model_cl_pred)
        print(confusion)
        print('accuracy : ', accuracy_score(test_slow_y,model_cl_pred))
        print('precision : ', precision_score(test_slow_y,model_cl_pred))
        print('recall : ', recall_score(test_slow_y,model_cl_pred))
        print('f1_score : ', f1_score(test_slow_y,model_cl_pred))
        print('')
    print('===================================================')
    print('')

LGBMClassifier()
threshold :  0.0
오차 행렬
[[    0 25545]
 [    0    30]]
accuracy :  0.0011730205278592375
precision :  0.0011730205278592375
recall :  1.0
f1_score :  0.0023432923257176333

threshold :  0.1
오차 행렬
[[25520    25]
 [    1    29]]
accuracy :  0.9989833822091887
precision :  0.5370370370370371
recall :  0.9666666666666667
f1_score :  0.6904761904761905

threshold :  0.2
오차 행렬
[[25521    24]
 [    1    29]]
accuracy :  0.9990224828934506
precision :  0.5471698113207547
recall :  0.9666666666666667
f1_score :  0.6987951807228916

threshold :  0.30000000000000004
오차 행렬
[[25523    22]
 [    1    29]]
accuracy :  0.9991006842619746
precision :  0.5686274509803921
recall :  0.9666666666666667
f1_score :  0.7160493827160493

threshold :  0.4
오차 행렬
[[25527    18]
 [    2    28]]
accuracy :  0.9992179863147606
precision :  0.6086956521739131
recall :  0.9333333333333333
f1_score :  0.7368421052631579

threshold :  0.5
오차 행렬
[[25530    15]
 [    2    28]]
accuracy :  0.999335288367546

## Feature Importance - Permutation

In [14]:
!pip install eli5

     |████████████████████████████████| 106 kB 5.4 MB/s 


In [15]:
import eli5
from eli5.sklearn import PermutationImportance
from eli5.lightgbm import *
from eli5.sklearn import *

### 급속

In [16]:
lgbm_fast = lgbm_clf_fast.fit(train_fast_X, train_fast_y)
perm = PermutationImportance(lgbm_fast, random_state=1).fit(test_fast_X, test_fast_y)
eli5.show_weights(perm, feature_names = test_fast_X.columns.tolist())

Weight,Feature
0.0004 ± 0.0001,급속
0.0004 ± 0.0001,면적_scaled
0.0004 ± 0.0000,완속_scaled
0.0004 ± 0.0000,위락시설_scaled
0.0003 ± 0.0001,교육연구및복지시설_scaled
0.0002 ± 0.0002,주거지_scaled
0.0002 ± 0.0000,개발부지_scaled
0.0002 ± 0.0000,청소년관련시설_scaled
0.0001 ± 0.0001,공장_scaled
0.0001 ± 0.0000,관광휴게시설_scaled


In [17]:
print(eli5.format_as_text(explain_weights.explain_permutation_importance(perm, feature_names = test_fast_X.columns.tolist(), top=40)))

explain_weights.explain_permutation_importance(perm, feature_names = test_fast_X.columns.tolist(), top=70)

Explained as: feature importances

Feature importances, computed as a decrease in score when feature
values are permuted (i.e. become noise). This is also known as 
permutation importance.

If feature importances are computed on the same data as used for training, 
they don't reflect importance of features for generalization. Use a held-out
dataset if you want generalization feature importances.

0.0004 ± 0.0001  급속
0.0004 ± 0.0001  면적_scaled
0.0004 ± 0.0000  위락시설_scaled
0.0004 ± 0.0000  완속_scaled
0.0003 ± 0.0001  교육연구및복지시설_scaled
0.0002 ± 0.0002  주거지_scaled
0.0002 ± 0.0000  개발부지_scaled
0.0002 ± 0.0000  청소년관련시설_scaled
0.0001 ± 0.0001  공장_scaled
0.0001 ± 0.0001  버스
0.0001 ± 0.0001  busStation
0.0001 ± 0.0000  관광휴게시설_scaled
0.0001 ± 0.0001  창고_scaled
0.0001 ± 0.0000  전기차_scaled
0.0001 ± 0.0000  종교시설
0.0001 ± 0.0000  width_1
0.0001 ± 0.0001  dw_lanes_0
0.0001 ± 0.0000  체력증진시설
0.0000 ± 0.0000  자동차관련시설_scaled
0.0000 ± 0.0001  혼잡시간강도_scaled
0.0000 ± 0.0000  통신시설_scaled
0.0000 ± 0.0001  산업단지거

Weight,Feature
0.0004 ± 0.0001,급속
0.0004 ± 0.0001,면적_scaled
0.0004 ± 0.0000,위락시설_scaled
0.0004 ± 0.0000,완속_scaled
0.0003 ± 0.0001,교육연구및복지시설_scaled
0.0002 ± 0.0002,주거지_scaled
0.0002 ± 0.0000,개발부지_scaled
0.0002 ± 0.0000,청소년관련시설_scaled
0.0001 ± 0.0001,공장_scaled
0.0001 ± 0.0000,관광휴게시설_scaled


### 완속

In [18]:
lgbm_slow = lgbm_clf_slow.fit(train_slow_X, train_slow_y)
perm = PermutationImportance(lgbm_slow, random_state=1).fit(test_slow_X, test_slow_y)
eli5.show_weights(perm, feature_names = test_slow_X.columns.tolist())

Weight,Feature
0.0005 ± 0.0001,노유자시설_scaled
0.0005 ± 0.0001,숙박시설_scaled
0.0005 ± 0.0003,종교시설
0.0003 ± 0.0001,교육연구및복지시설_scaled
0.0002 ± 0.0001,width_5_scaled
0.0002 ± 0.0001,공공시설_scaled
0.0001 ± 0.0000,dw_lanes_3
0.0001 ± 0.0001,유원지
0.0001 ± 0.0000,주차장_scaled
0.0001 ± 0.0001,면적_scaled


In [19]:
print(eli5.format_as_text(explain_weights.explain_permutation_importance(perm, feature_names = test_slow_X.columns.tolist(), top=40)))

explain_weights.explain_permutation_importance(perm, feature_names = test_slow_X.columns.tolist(), top=70)

Explained as: feature importances

Feature importances, computed as a decrease in score when feature
values are permuted (i.e. become noise). This is also known as 
permutation importance.

If feature importances are computed on the same data as used for training, 
they don't reflect importance of features for generalization. Use a held-out
dataset if you want generalization feature importances.

0.0005 ± 0.0001  노유자시설_scaled
0.0005 ± 0.0001  숙박시설_scaled
0.0005 ± 0.0003  종교시설
0.0003 ± 0.0001  교육연구및복지시설_scaled
0.0002 ± 0.0001  width_5_scaled
0.0002 ± 0.0001  공공시설_scaled
0.0001 ± 0.0000  dw_lanes_3
0.0001 ± 0.0001  유원지
0.0001 ± 0.0000  주차장_scaled
0.0001 ± 0.0001  면적_scaled
0.0001 ± 0.0000  차량수_scaled
0.0001 ± 0.0000  혼잡빈도강도
0.0001 ± 0.0000  급속
0.0001 ± 0.0000  문화및집회시설_scaled
0.0001 ± 0.0000  창고_scaled
0.0001 ± 0.0002  화물차
0.0001 ± 0.0001  혼잡시간강도_scaled
0.0000 ± 0.0000  위락시설_scaled
0.0000 ± 0.0000  up_lanes_6_scaled
0.0000 ± 0.0000  학교_scaled
0.0000 ± 0.0000  운수시설_scaled
0.0000 ± 0.0001  

Weight,Feature
0.0005 ± 0.0001,노유자시설_scaled
0.0005 ± 0.0001,숙박시설_scaled
0.0005 ± 0.0003,종교시설
0.0003 ± 0.0001,교육연구및복지시설_scaled
0.0002 ± 0.0001,width_5_scaled
0.0002 ± 0.0001,공공시설_scaled
0.0001 ± 0.0000,dw_lanes_3
0.0001 ± 0.0001,유원지
0.0001 ± 0.0000,주차장_scaled
0.0001 ± 0.0001,면적_scaled


## Feature Importance

In [20]:
fea_imp_ = pd.DataFrame({'cols':train_fast_X.columns, 'fea_imp':lgbm_clf_fast.feature_importances_})
fea_imp_.sort_values(by=['fea_imp'], ascending=True)

,cols,fea_imp
44,dw_lanes_5_scaled,0
42,up_lanes_5_scaled,0
41,up_lanes_6_scaled,0
62,유원지,0
40,width_5_scaled,0
...,...,...
64,체력증진시설,97
8,주거지_scaled,98
59,급속,135
25,면적_scaled,154


## Optuna

In [21]:
!pip install optuna

     |████████████████████████████████| 308 kB 5.3 MB/s 
     |████████████████████████████████| 81 kB 8.6 MB/s 
     |████████████████████████████████| 210 kB 43.0 MB/s 
     |████████████████████████████████| 75 kB 4.7 MB/s 
     |████████████████████████████████| 113 kB 46.1 MB/s 
     |████████████████████████████████| 49 kB 5.7 MB/s 
     |████████████████████████████████| 150 kB 42.9 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=25982b58346929d6d03b613c7a5e63b24ed29dcdaa2d3c8e64c5942136c7c3c2
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [22]:
import pandas as pd
import numpy as np
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [23]:
X_fast_train = train_fast.iloc[:, :-1]
y_fast_train = train_fast.iloc[:, -1]
X_fast_test = test_fast.iloc[:, :-1]
y_fast_test = test_fast.iloc[:, -1]

X_slow_train = train_slow.iloc[:, :-1]
y_slow_train = train_slow.iloc[:, -1]
X_slow_test = test_slow.iloc[:, :-1]
y_slow_test = test_slow.iloc[:, -1]

In [62]:
def LGBM_objective(trial):
    base_score = 0.5
    learning_rate = trial.suggest_float('learning_rate', 0, 1)
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', -1, 50)
    boosting = trial.suggest_categorical('boosting', ['rf', 'gbdt', 'dart']) #goss
    metric = 'binary_logloss'

    feature_fraction = trial.suggest_float('feature_fraction', 0.6, 0.9)
    scale_pos_weight = trial.suggest_float('scale_pos_weight', 1, 1.7)

    num_leaves = trial.suggest_int('num_leaves', 2, 40)
    bagging_freq = trial.suggest_int('bagging_freq', 0, 50)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0, 1)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 50, 1000)

    n_jobs = -1
    random_state = 42
    objective = 'binary'
    subsample = 1
    #device='gpu'
    
   
    model = LGBMClassifier(base_score=base_score, boosting=boosting, learning_rate=learning_rate, max_depth=max_depth, n_jobs=n_jobs, random_state=random_state, subsample=subsample, 
                           n_estimators=n_estimators, metric=metric, feature_fraction=feature_fraction, scale_pos_weight=scale_pos_weight,
                           num_leaves=num_leaves, bagging_freq=bagging_freq, bagging_fraction=bagging_fraction, min_data_in_leaf=min_data_in_leaf,
                           objective=objective, 
                           #device=device
                          )

    model.fit(X_fast_train, y_fast_train)
    y_fast_pred = model.predict(X_fast_test)
    f1 = f1_score(y_fast_test, y_fast_pred)

    return f1
    
#Execute optuna and set hyperparameters
LGBM_study = optuna.create_study(direction='maximize')
LGBM_study.optimize(LGBM_objective, n_trials=100)


#Create an instance with tuned hyperparameters
optimized_LGBM = LGBMClassifier(base_score=LGBM_study.best_params['base_score'], boosting=LGBM_study.best_params['boosting'], learning_rate=LGBM_study.best_params['learning_rate'],
                                max_depth=LGBM_study.best_params['max_depth'], n_estimators=LGBM_study.best_params['n_estimators'], feature_fraction=LGBM_study.best_params['feature_fraction'],
                                scale_pos_weight=LGBM_study.best_params['scale_pos_weight'], num_leaves=LGBM_study.best_params['num_leaves'], bagging_freq=LGBM_study.best_params['bagging_freq'],
                                bagging_fraction=LGBM_study.best_params['bagging_fraction'], min_data_in_leaf=LGBM_study.best_params['min_data_in_leaf'],
                                objective='binary', n_jobs=-1, random_state=42, subsample=1, metric='binary_logloss',
                                #device='gpu'
                               )

[I 2022-02-25 08:19:09,008] A new study created in memory with name: no-name-02091bf3-979f-4861-97de-92eb77ffb4ac
[I 2022-02-25 08:19:55,771] Trial 0 finished with value: 0.7659574468085107 and parameters: {'learning_rate': 0.9751701893149586, 'n_estimators': 388, 'max_depth': 40, 'boosting': 'dart', 'feature_fraction': 0.6615495016932255, 'scale_pos_weight': 1.0767034869457708, 'num_leaves': 32, 'bagging_freq': 13, 'bagging_fraction': 0.8707092518397507, 'min_data_in_leaf': 751}. Best is trial 0 with value: 0.7659574468085107.
[I 2022-02-25 08:20:12,762] Trial 1 finished with value: 0.06512890094979647 and parameters: {'learning_rate': 0.199498768419197, 'n_estimators': 941, 'max_depth': 19, 'boosting': 'rf', 'feature_fraction': 0.6299811768119143, 'scale_pos_weight': 1.0155059352564053, 'num_leaves': 30, 'bagging_freq': 47, 'bagging_fraction': 0.36739764754843773, 'min_data_in_leaf': 526}. Best is trial 0 with value: 0.7659574468085107.
[I 2022-02-25 08:21:03,205] Trial 2 finished wi

LightGBMError: ignored

[I 2022-02-25 08:21:41,264] Trial 5 finished with value: 0.8000000000000002 and parameters: {'learning_rate': 0.24126919501069777, 'n_estimators': 113, 'max_depth': 31, 'boosting': 'gbdt', 'feature_fraction': 0.8637387162156923, 'scale_pos_weight': 1.6314529692749955, 'num_leaves': 33, 'bagging_freq': 16, 'bagging_fraction': 0.8965246003392066, 'min_data_in_leaf': 890}. Best is trial 5 with value: 0.8000000000000002.

In [63]:
def LGBM_objective(trial):
    base_score = 0.5
    learning_rate = trial.suggest_float('learning_rate', 0, 1)
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', -1, 50)
    boosting = trial.suggest_categorical('boosting', ['rf', 'gbdt', 'dart']) #goss
    metric = 'binary_logloss'

    feature_fraction = trial.suggest_float('feature_fraction', 0.6, 0.9)
    scale_pos_weight = trial.suggest_float('scale_pos_weight', 1, 1.7)

    num_leaves = trial.suggest_int('num_leaves', 2, 40)
    bagging_freq = trial.suggest_int('bagging_freq', 1, 50)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.01, 0.99)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 50, 1000)

    n_jobs = -1
    random_state = 42
    objective = 'binary'
    subsample = 1
    #device='gpu'
    
   
    model = LGBMClassifier(base_score=base_score, boosting=boosting, learning_rate=learning_rate, max_depth=max_depth, n_jobs=n_jobs, random_state=random_state, subsample=subsample, 
                           n_estimators=n_estimators, metric=metric, feature_fraction=feature_fraction, scale_pos_weight=scale_pos_weight,
                           num_leaves=num_leaves, bagging_freq=bagging_freq, bagging_fraction=bagging_fraction, min_data_in_leaf=min_data_in_leaf,
                           objective=objective, 
                           #device=device
                          )

    model.fit(X_fast_train, y_fast_train)
    y_fast_pred = model.predict(X_fast_test)
    f1 = f1_score(y_fast_test, y_fast_pred)

    return f1
    
#Execute optuna and set hyperparameters
LGBM_study = optuna.create_study(direction='maximize')
LGBM_study.optimize(LGBM_objective, n_trials=100)


#Create an instance with tuned hyperparameters
optimized_LGBM = LGBMClassifier(base_score=LGBM_study.best_params['base_score'], boosting=LGBM_study.best_params['boosting'], learning_rate=LGBM_study.best_params['learning_rate'],
                                max_depth=LGBM_study.best_params['max_depth'], n_estimators=LGBM_study.best_params['n_estimators'], feature_fraction=LGBM_study.best_params['feature_fraction'],
                                scale_pos_weight=LGBM_study.best_params['scale_pos_weight'], num_leaves=LGBM_study.best_params['num_leaves'], bagging_freq=LGBM_study.best_params['bagging_freq'],
                                bagging_fraction=LGBM_study.best_params['bagging_fraction'], min_data_in_leaf=LGBM_study.best_params['min_data_in_leaf'],
                                objective='binary', n_jobs=-1, random_state=42, subsample=1, metric='binary_logloss',
                                #device='gpu'
                               )

[I 2022-02-25 08:46:36,022] A new study created in memory with name: no-name-76c0c4ad-ea41-4f0b-958c-f815ae1d2555
[I 2022-02-25 08:46:37,832] Trial 0 finished with value: 0.03856266432953549 and parameters: {'learning_rate': 0.45871547831524395, 'n_estimators': 191, 'max_depth': 4, 'boosting': 'gbdt', 'feature_fraction': 0.7028589105158327, 'scale_pos_weight': 1.3512373490181395, 'num_leaves': 15, 'bagging_freq': 6, 'bagging_fraction': 0.08850161289334506, 'min_data_in_leaf': 136}. Best is trial 0 with value: 0.03856266432953549.
[I 2022-02-25 08:46:43,525] Trial 1 finished with value: 0.0023954703832752616 and parameters: {'learning_rate': 0.48415746329515663, 'n_estimators': 373, 'max_depth': 0, 'boosting': 'gbdt', 'feature_fraction': 0.6268597997979648, 'scale_pos_weight': 1.4923668513235093, 'num_leaves': 8, 'bagging_freq': 36, 'bagging_fraction': 0.7098542377063976, 'min_data_in_leaf': 797}. Best is trial 0 with value: 0.03856266432953549.
[I 2022-02-25 08:47:19,315] Trial 2 finis

KeyError: ignored

In [64]:
optuna.visualization.plot_param_importances(LGBM_study)

In [65]:
optuna.visualization.plot_optimization_history(LGBM_study)



---

